In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ml_models import models_dict
from sklearn.manifold import TSNE

In [2]:
def data_preprocessor(fname):
    with open(fname, 'r') as f:
        rules = f.read().split('\n')

    X,  Y = [], []
    for rule in rules:
        (UA, OA, P) = rule.split(';')
        UA = list(map(int, UA.split(',')))
        OA = list(map(int, OA.split(',')))
        X.append(UA + OA)
        Y.append(int(P))
    return (np.array(X), np.array(Y))

In [3]:
def tsne_plot(x_reduced, x_predicted, clf):
    x_min, x_max = x_reduced[:, 0].min() - 5, x_reduced[:, 0].max() + 5
    y_min, y_max = x_reduced[:, 1].min() - 5, x_reduced[:, 1].max() + 5

    x_ = np.linspace(x_min, x_max, 500)
    y_ = np.linspace(y_min, y_max, 500)

    xx, yy = np.meshgrid(x_, y_)

    # evaluate the decision function on the meshgrid
    try:
        z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        z = z.reshape(xx.shape)
    except AttributeError:
        return

    # plot the decision function and the reduced data
    plt.contourf(xx, yy, z)
    a = plt.contour(xx, yy, z, levels=[0], linewidths=1.5, colors='magenta')
    b = plt.scatter(x_reduced[x_predicted==1, 0], x_reduced[x_predicted==1, 1], c='green', marker='+')
    c = plt.scatter(x_reduced[x_predicted==0, 0], x_reduced[x_predicted==0, 1], c='red', marker='+')
    plt.legend([a.collections[0], b, c], ['decision boundary', '+ve', '-ve'], bbox_to_anchor=(1.05, 1))
    plt.axis('tight')
    plt.show()

In [4]:
Xtrain, Ytrain = data_preprocessor("abac-cat-v1.txt")
Xtest, Ytest = data_preprocessor("test-v1.txt")

In [5]:
# reduce the dimensions of the train and test data using TSNE
size_train = Xtrain.shape[0]
X = np.concatenate((Xtrain, Xtest), axis=0)
X_tsne = TSNE(n_components=2, n_jobs=-1).fit_transform(X) 
Xtrain = X_tsne[:size_train]
Xtest = X_tsne[size_train:]

In [ ]:
for (name, clf) in models_dict.items():
    print(f"[INFO] Training model: {name}")
    clf.fit(Xtrain, Ytrain)
    pred = clf.predict(Xtest)
    tsne_plot(Xtest, pred, clf)

[INFO] Training model: SupportVectorMachine
